## How to create a custom executor

<font size="3">


Each electron can utilise different so-called executors. These executors define the low-level directions for the computation. They can specify different capabilities, _eg_, different hardware, different computation strategy, different logic, or simply different goals.

Executors are plugins; any executor-plugins which are found are imported as classes in the covalent.executor name-space.
See the how-to on [choosing an executor to be used in an electron](choosing_executors.ipynb) for details on simply choosing an executor. Here, we go through the steps, line-by-line, on how to make your own.
</font>

<font size="3">

There is a certain amount of boiler-plate Covalent code and imports that are required:
</font>

```python
# All executor plugins inherit from the BaseExecutor base class. The wrapper_fn is used to wrap the TransportableObject between the
# call_before and call_after hooks
from covalent.executor import BaseExecutor, wrapper_fn

# The current status of the execution can be kept up-to-date with Covalent Result objects.
from covalent._results_manager.result import Result

# DispatchInfo objects are used to share info of a dispatched computation between different
# tasks (electrons) of the workflow (lattice).
from covalent._shared_files.util_classes import DispatchInfo

# The function to be computed is in the form of a Covalent TransportableObject.
# This import is not strictly necessary, as it is only used for type-hints.
from covalent._workflow.transport import TransportableObject

# The Covalent logger module is a simple wrapper around the
# standard Python logging module.
from covalent._shared_files import logger
app_log = logger.app_log
log_stack_info = logger.log_stack_info
```

<font size="3">

We also need a few required standard Python imports:
</font>

```python
# These modules are used to capture print/logging statements
# inside electron definitions.
import io
from contextlib import redirect_stderr, redirect_stdout

# For type-hints
from typing import Any, Dict, List
```

<font size="3">
Somewhere in the module, the module attribute "EXECUTOR_PLUGIN_NAME" must be set to the class name defining the executor:
</font>

```python
# The plugin class name must be given by the EXECUTOR_PLUGIN_NAME attribute. In case this
# module has more than one class defined, this lets Covalent know which is the executor class.
EXECUTOR_PLUGIN_NAME = "CustomExecutor"
```

<font size="3">

Executor-specific inputs should have sane defaults defined in the global variable `_EXECUTOR_PLUGIN_DEFAULTS`. These defaults will be written to the user's Covalent configuration file, if they were not already present.
</font>

```python 
_EXECUTOR_PLUGIN_DEFAULTS = {
    "executor_input1": "",
    "executor_input2": "",
}
```

<font size="3">

Now we define the executor plugin class. Input arguments to the executor class are not necessary, but in this example we have two, as well as other arbitrary arguments (*args) and keyword arguments (**kwargs).
</font>

```python
class CustomExecutor(BaseExecutor):
    def __init__(
        # Inputs to an executor can be positional or keyword arguments.
        self,
        executor_input1: str,
        executor_input2: int = 0,
        *args,
        **kwargs,

    ) -> None:
        self.executor_input1 = executor_input1
        self.executor_input2 = executor_input2
        self.kwargs = kwargs

        # Call the BaseExecutor initialization:
        # There are a number of optional arguments to BaseExecutor
        # that could be specfied as keyword inputs to CustomExecutor.
        base_kwargs = {}
        for key, _ in self.kwargs.items():
            if key in [
                "conda_env",
                "cache_dir",
                "current_env_on_conda_fail",
            ]:
                base_kwargs[key] = self.kwargs[key]

        super().__init__(**base_kwargs)
```

<font size="3">

The "execute" class method is the main work-horse of the executor plugin. Here is the definition of where and how the input function will be computed.
</font>

```python
    def execute(
        self,
        function: TransportableObject,
        args: List,
        kwargs: Dict,
        call_before: List,
        call_after: List,
        dispatch_id: str,
        results_dir: str,
        node_id: int = -1
    ) -> Any:

        """
        Execute the function with the given arguments. The `call_before` and `call_after` are a
        list of callables that are executed before and after the function execution respectively

        Args:
            function: The input (serialized) python function which will be executed and
                whose result is ultimately returned by this function.
            args: List of positional arguments to be used by the function.
            kwargs: Dictionary of keyword arguments to be used by the function.
            call_before: List of Python callables to be executed before function executes.
            call_after: List of Python callables to be executed after function executes.
            dispatch_id: The unique identifier of the external lattice process which is calling the function.
            results_dir: The location of the results directory.
            node_id: The ID of this task/node in the workflow graph.

        Returns:
            output: The result of the executed function.
        """
```

<font size="3">

Here, we convert the function from a Covalent TransportableObject to a "standard" Python function. From here, we are free to create whatever operations make this custom executor unique.
</font>

```python
        # Convert the function to be executed, which is in the form of
        # a Covalent TransportableObject, to a 'standard' Python function.
        # Invoke the wrapper_fn to get a transportable object with call_before/call_after hooks placed properly
        fn = wrapper_fn(function, call_before, call_after, args, kwargs).get_deserialized()
        app_log.debug(type(fn))

        # In this block is where operations specific to your custom executor
        # can be defined. These operations could manipulate the function, the
        # inputs/outputs, or anything that Python allows.
        external_object = ExternalClass(3)
        app_log.debug(external_object.multiplier)
        
        # Store the current status to this shared-between-processes object.
        info_dict = {"STATUS": Result.RUNNING}
        info_queue.put_nowait(info_dict)
```

<font size="3">

The execution of the input function must be done within the following context manager:
</font>

```python
        result = None
        exception = None

        with self.get_dispatch_context(DispatchInfo(dispatch_id)), redirect_stdout(
            io.StringIO()
        ) as stdout, redirect_stderr(io.StringIO()) as stderr:
            # Here we simply execute the function on the local machine.
            # But this could be sent to a more capable machine for the operation,
            # or a different Python virtual environment, or more.
            try:
                result = fn(*args, **kwargs)
            except Exception as e:
                exception = e
```

<font size="3">

Other unique custom operations or informational logging can be done here as well.
</font>

```python
        # Other custom operations can be applied here.
        if result is not none:
            result = self.helper_function(result)

        debug_message = f"Function '{fn.__name__}' was executed on node {node_id} with execution arguments {execution_args}"
        app_log.debug(debug_message)
        
        # Update the status:
        info_dict = info_queue.get()
        info_dict["STATUS"] = Result.FAILED if result is None else Result.COMPLETED
        info_queue.put(info_dict)
```

<font size="3">

Finally, return the result and any print statements or log/error messages from the execution, if any:
</font>

```python
        return (result, stdout.getvalue(), stderr.getvalue(), exception)
```

<font size="3">

Other class fucntions can, of course, be defined and used:
</font>

```python
    def helper_function(self, result):
        return 2*result
    
    def get_status(self, info_dict: dict) -> Result:
        """
        Get the current status of the task.

        Args:
            info_dict: a dictionary containing any neccessary parameters needed to query the
                status. For this class (LocalExecutor), the only info is given by the
                "STATUS" key in info_dict.

        Returns:
            A Result status object (or None, if "STATUS" is not in info_dict).
        """

        return info_dict.get("STATUS", Result.NEW_OBJ)
    
    def cancel(self, info_dict: dict = {}) -> Tuple[Any, str, str]:
        """
        Cancel the execution task.

        Args:
            info_dict: a dictionary containing any neccessary parameters
                needed to halt the task execution.

        Returns:
            Null values in the same structure as a successful return value (a 4-element tuple).
        """

        return (None, "", "", InterruptedError)
```

<font size="3">

Similarly, other classes can be in the same module:
</font>

```python
# This class can be used in the custom executor, but will be ignored by the
# plugin-loader, since it is not designated as the plugin class.
class ExternalClass:

    def __init__(self, multiplier: int):
        self.multiplier = multiplier
```

<font size="3">

Here is the entire module in one executable cell:
</font>

In [1]:
# Copyright 2021 Agnostiq Inc.
#
# This file is part of Covalent.
#
# Licensed under the GNU Affero General Public License 3.0 (the "License").
# A copy of the License may be obtained with this software package or at
#
#      https://www.gnu.org/licenses/agpl-3.0.en.html
#
# Use of this file is prohibited except in compliance with the License. Any
# modifications or derivative works of this file must retain this copyright
# notice, and modified files must contain a notice indicating that they have
# been altered from the originals.
#
# Covalent is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
# FITNESS FOR A PARTICULAR PURPOSE. See the License for more details.
#
# Relief from the License may be granted by purchasing a commercial license.

"""This is an example of a custom Covalent executor plugin."""

# The Covalent logger module is a simple wrapper around the
# These modules are used to capture print/logging statements
# inside electron definitions.
import io
from contextlib import redirect_stderr, redirect_stdout

# For type-hints
from typing import Any, Dict, List, Tuple

# The current status of the execution can be kept up-to-date with Covalent Result objects.
from covalent._results_manager.result import Result

# Covalent logger
from covalent._shared_files import logger

# DispatchInfo objects are used to share info of a dispatched computation between different
# tasks (electrons) of the workflow (lattice).
from covalent._shared_files.util_classes import DispatchInfo

# The function to be computed is in the form of a Covalent TransportableObject.
# This import is not strictly necessary, as it is only used for type-hints.
from covalent._workflow.transport import TransportableObject

# All executor plugins inherit from the BaseExecutor base class.
from covalent.executor import BaseExecutor, wrapper_fn

app_log = logger.app_log
log_stack_info = logger.log_stack_info

# The plugin class name must be given by the EXECUTOR_PLUGIN_NAME attribute. In case this
# module has more than one class defined, this lets Covalent know which is the executor class.
EXECUTOR_PLUGIN_NAME = "CustomExecutor"

_EXECUTOR_PLUGIN_DEFAULTS = {
    "executor_input1": "",
    "executor_input2": "",
}


class CustomExecutor(BaseExecutor):
    def __init__(
        # Inputs to an executor can be positional or keyword arguments.
        self,
        executor_input1: str,
        executor_input2: int = 0,
        **kwargs,
    ) -> None:
        self.executor_input1 = executor_input1
        self.executor_input2 = executor_input2
        self.kwargs = kwargs

        # Call the BaseExecutor initialization:
        # There are a number of optional arguments to BaseExecutor
        # that could be specfied as keyword inputs to CustomExecutor.
        base_kwargs = {}
        for key, _ in self.kwargs.items():
            if key in [
                "conda_env",
                "cache_dir",
                "current_env_on_conda_fail",
            ]:
                base_kwargs[key] = self.kwargs[key]

        super().__init__(**base_kwargs)

    def execute(
        self,
        function: TransportableObject,
        args: List,
        kwargs: Dict,
        call_before: List,
        call_after: List,
        dispatch_id: str,
        results_dir: str,
        node_id: int = -1
    ) -> Any:

        """
        Executes the input function and returns the result.

        Args:
            function: The input (serialized) python function which will be executed and
                whose result is ultimately returned by this function.
            args: List of positional arguments to be used by the function.
            kwargs: Dictionary of keyword arguments to be used by the function.
            call_before: List of Python callables to be executed before function executes.
            call_after: List of Python callables to be executed after function executes.
            dispatch_id: The unique identifier of the external lattice process which is calling the function.
            results_dir: The location of the results directory.
            node_id: The ID of this task/node in the workflow graph.

        Returns:
            output: The result of the executed function.
        """

        # Convert the function to be executed, which is in the form of
        # a Covalent TransportableObject, to a 'standard' Python function:

        fn = wrapper_fn(function, call_before, call_after, args, kwargs).get_deserialized()
        app_log.debug(type(fn))

        # In this block is where operations specific to your custom executor
        # can be defined. These operations could manipulate the function, the
        # inputs/outputs, or anything that Python allows.
        external_object = ExternalClass(3)
        app_log.debug(external_object.multiplier)

        result = None
        exception = None

        with self.get_dispatch_context(DispatchInfo(dispatch_id)), redirect_stdout(
            io.StringIO()
        ) as stdout, redirect_stderr(io.StringIO()) as stderr:
            # Here we simply execute the function on the local machine.
            # But this could be sent to a more capable machine for the operation,
            # or a different Python virtual environment, or more.
            try:
                result = fn(*args, **kwargs)
            except Exception as e:
                exception = e

        # Other custom operations can be applied here.
        if result is not None:
            result = self.helper_function(result)

        debug_message = f"Function '{fn.__name__}' was executed on node {node_id}"
        app_log.debug(debug_message)

        return (result, stdout.getvalue(), stderr.getvalue(), exception)

    def helper_function(self, result):
        """An example helper function."""

        return 2 * result

    def get_status(self, info_dict: dict) -> Result:
        """
        Get the current status of the task.

        Args:
            info_dict: a dictionary containing any neccessary parameters needed to query the
                status. For this class (LocalExecutor), the only info is given by the
                "STATUS" key in info_dict.

        Returns:
            A Result status object (or None, if "STATUS" is not in info_dict).
        """

        return info_dict.get("STATUS", Result.NEW_OBJ)

    def cancel(self, info_dict: dict = {}) -> Tuple[Any, str, str]:
        """
        Cancel the execution task.

        Args:
            info_dict: a dictionary containing any neccessary parameters
                needed to halt the task execution.

        Returns:
            Null values in the same structure as a successful return value (a 4-element tuple).
        """

        return (None, "", "", InterruptedError)


# This class can be used in the custom executor, but will be ignored by the
# plugin-loader, since it is not designated as the plugin class.
class ExternalClass:
    """An example external class."""

    def __init__(self, multiplier: int):
        self.multiplier = multiplier


### Example

Using the aforementioned steps, let's create a Docker executor that can be used in workflows to execute electrons inside containers. NOTE: Covalent already has a `DockerExecutor` plugin that can be installed with `covalent` via `pip install covalent-docker-plugin`.

We will implement this executor in stages in order to better illustrate the steps invovled and as well as to facilitate understanding. The idea behind creating a `DockerExecutor` is to allow user's to run their electrons in a containerized environment. As a starting step, we first create a docker image that we will use later to launch containers and run electrons within. We use the following `Dockerfile` for this purpose and install the necessary dependencies within.

```docker
# Dockerfile
FROM python:3.8-buster

RUN pip install cloudpickle && \
    pip install covalent

CMD ["/bin/bash"]
```

In this Dockerfile, we install the bare minimum dependencies required to execute electrons. Next we build an image `docker-executor-demo:latest` from the Dockerfile template to be used in our workflows.

```shell
docker build -f Dockerfile --tag docker-executor-demo:latest .
```

The approach we intend to take in developing this executor is the following:

* When the executor's execute method is called, we create a container using the `image` provided by the user as part of executor instantiation
* As part of the container run command, we bind mount the user specified working directory into the container's working directory in `rw` mode. This is to allow retriving of any files generated within the container as part of the function execution to be accessible from outside of it before/during and after container run
* Once the container is running, we using `docker container exec` to execute the pickled function object inside the container
* Assert the `result` file gets created in the user's working directory as an artifact, if not raise an exception
* Read the result object from the generated result file into `result`
* Stop the container
* Return `result`

#### Step 1

We can now being writing our `DockerExecutor` by first importing the boilerplate as well as any other dependencies needed by the executor.

```python
import os
import copy
import shutil
import cloudpickle as pickle
import tempfile
import subprocess
from pathlib import Path
from typing import List, Dict

from covalent.executor import BaseExecutor, wrapper_fn
from covalent._shared_files.util_classes import DispatchInfo
from covalent._shared_files import logger
from covalent._workflow.transport import TransportableObject
```

#### Step 2

To enable logging, we use the loggers present in Covalent itself

```python
app_log = logger.app_log
log_stack_info = logger.log_stack_info
```

#### Step 3

We now specify any default configuration options for this executor. Note that specifying the configuration is entirely dependent on the author the the executor, as they can choose to expose any number of configuration options for the executors here.

```python
_EXECUTOR_PLUGIN_DEFAULTS = {
    "image": "python:3.8"   # Default docker image to be used if not specified
    "workdir": "." # Path to the working directory on disk where files generated during execution will be        stored. Defaults to current working directory
    "options": {}    # Any options supported by the docker run CLI
}
```

#### Step 4

Set the executor plugin name

```python
executor_plugin_name = 'DockerExecutor'
```

#### Step 5

Create the `DockerExecutor` class while deriving it from the `BaseExecutor` imported earlier.


```python
class DockerExecutor(BaseExecutor):
    """Docker executor plugin class
    
    Args:
        :param str image: Name of the docker image to be used for running electrons.
        :param str workdir: Path on the disk where files generated during execution will be created/stored.
        :param dict options: Python dictionary of keyword arguments of the different options supported by the `docker run` CLI.
    """

    def __init__(self,
        image: str,
        workdir: str,
        options: Dict = {}
        container_workdir: str = "/tmp/covalent",  # Workdir inside the container
        *args,
        **kwargs
    ):

        self.image = image
        self.wordir = workdir
        self.options = options
        self._container_workdir = container_workdir

        super().__init__(*args, **kwargs)
```

#### Step 6

As seen from the outline, the executor is quite simple in the sense that it starts a container using th image provided by the user, bind mounts the working directory into the container, executes a command inside the running container and returns the result.

The command to be executed can be rendered as a multiline string. To this end, we create a method `_format_exec_command` in our `DockerExecutor` to facilitate generation of the `exec` command

```python
def _format_exec_command(self, func_filename: str, result_filename: str) -> str:
    """
    Create a python string that can be used to execute the task inside the container

    Args:
        :param str func_filename: Name of the pickle file from which to read the function to be executed.
        :param str result_filename: Name of the pickle file into which the serialize and write the task result

    Returns:
        :param str script: Python string that can be parsed to execute the function
    """
    return "\n".join(
        [
            "import cloudpickle as pickle",
            "",
            "result = None",
            "exception = None",
            "",
            "with open('{func_filename}', 'rb') as f:",
            "   function, args, kwargs = pickle.load(f)",
            "   try:",
            "       result = function(*args, **kwargs)",
            "   except Exception as e:",
            "       exception = e",
            "   finally:",
            "       with open('{result_filename}', 'wb') as f_out:",
            "           pickle.dump((result, exception), f_out)",
            "",
        ]
    ).format(
        func_filename = os.path.join(self._container_workdir, func_filename),
        result_filename = os.path.join(self._container_workdir, result_filename)
    )
```


#### Step 7

With the `_format_exec_command` helper method defined, we can now implement the executor's `execute` method that contains the core logic

```python
def execute(self,
    function: TransportableObject,
    args: List,
    kwargs: List,
    call_before: List,
    call_after: List,
    dispatch_id: str,
    results_dir: str,
    node_id: int - 1
):
    """
        Execute the callable inside the respective container and return the result

        Args:
            :param Callable function: The input python function to be executed inside the container whose result is to be returned back
            :param List args: List of positional arguments required by the function
            :param Dict kwargs: Dictionary of keyword arguments required by the function
            :param List call_before: List of callables that will be invoked before the actual function execution
            :param List  call_after: List of callables to be invoked after the function has finished execution
            :param str dispatch_id: The unique identifier of the parent workflow that this electron is a part of
            :param str results_dir: The location where the results from the function execution ought to be saved. This will be mounted inside the container
            :param int node_id: ID of the node in the transport graph which is using this executor
    """

    # set the necessary variables such as filenames to be used as part of the execution.
    # Use the dispatch_id and the node_id in order to uniquely identify each file/container being
    # created during execution
    dispatch_info = DispatchInfo(dispatch_id=dispatch_id)
    result_filename = f"result-{dispatch_id}-{node_id}.pkl"
    container_name = f"container-{dispatch_id}-{node_id}"
    execute_script_name = f"dkrexec-{dispatch_id}-{node_id}.py"

    with self.get_dispatch_context(dispatch_info), tempfile.NamedTemporaryFile(
            dir=self.workdir) as f, tempfile.NamedTemporaryFile(dir=self.workdir, mode="w") as g:

            # Wrap the `function` between the call_before and call_after hooks
            new_args = [function, call_before, call_after]
            for arg in args:
                new_args.append(arg)

            pickle.dump((wrapper_fn, new_args, kwargs), f)
            f.flush()

            # Format the command to be executed inside the container
            func_filename = f"func-{dispatch_id}-{node_id}.pkl"
            shutil.copy(f.name, os.path.join(self.workdir, func_filename))

            cmd = self.format_exec_command(func_filename, result_filename)
            g.write(cmd)
            g.flush()
            shutil.copy(g.name, os.path.join(self.workdir, execute_script_name))

            # Start the container in detached mode
            subprocess.run(
                [
                    "docker",
                    "container",
                    "run",
                    "-dit",
                     "--rm",
                    "--mount",
                    f"type=bind,source={self.workdir},target={self._container_workdir}",
                    "--name",
                    container_name,
                    self.image
                ],
                check=True,
                capture_output=True,
            )

            # Execute the script/command inside the container
            subprocess.run(
                [
                    "docker",
                    "container",
                    "exec",
                    container_name,
                    "python",
                    f"{self._container_workdir}/{execute_script_name}"
                ],
                check=True,
                capture_output=True
            )

            # Assert that a result object was created
            assert os.path.exists(os.path.join(self.workdir, result_filename))

            # Read the generated result object from the result pickle file
            with open(os.path.join(self.workdir, result_filename), "rb") as f_read:
                result, exception = pickle.load(f_read)
                if exception:
                    raise exception

            # Terminate the container
            subprocess.run(
                [
                    "docker",
                    "container",
                    "stop",
                    container_name
                ],
                check=True,
                capture_output=True
            )

            # Return
            return result, exception, None
```

For convenience, here is the entire executor as part of a single executable block

```python
import os
import copy
import shutil
import cloudpickle as pickle
import tempfile
import subprocess
from pathlib import Path
from typing import List, Dict

from covalent.executor import BaseExecutor, wrapper_fn
from covalent._shared_files.util_classes import DispatchInfo
from covalent._shared_files import logger
from covalent._workflow.transport import TransportableObject

app_log = logger.app_log
log_stack_info = logger.log_stack_info

_EXECUTOR_PLUGIN_DEFAULTS = {
    "image": "python:3.8"   # Default docker image to be used if not specified
    "workdir": "." # Path to the working directory on disk where files generated during execution will be        stored. Defaults to current working directory
    "options": {}    # Any options supported by the docker run CLI
}

executor_plugin_name = 'DockerExecutor'

class DockerExecutor(BaseExecutor):
    """Docker executor plugin class
    
    Args:
        :param str image: Name of the docker image to be used for running electrons.
        :param str workdir: Path on the disk where files generated during execution will be created/stored.
        :param dict options: Python dictionary of keyword arguments of the different options supported by the `docker run` CLI.
    """

    def __init__(self,
        image: str,
        workdir: str,
        options: Dict = {}
        container_workdir: str = "/tmp/covalent",  # Workdir inside the container
        *args,
        **kwargs
    ):

        self.image = image
        self.wordir = workdir
        self.options = options
        self._container_workdir = container_workdir

        super().__init__(*args, **kwargs)

    def _format_exec_command(self, func_filename: str, result_filename: str) -> str:
    """
    Create a python string that can be used to execute the task inside the container

    Args:
        :param str func_filename: Name of the pickle file from which to read the function to be executed.
        :param str result_filename: Name of the pickle file into which the serialize and write the task result

    Returns:
        :param str script: Python string that can be parsed to execute the function
    """
        return "\n".join(
            [
                "import cloudpickle as pickle",
                "",
                "result = None",
                "exception = None",
                "",
                "with open('{func_filename}', 'rb') as f:",
                "   function, args, kwargs = pickle.load(f)",
                "   try:",
                "       result = function(*args, **kwargs)",
                "   except Exception as e:",
                "       exception = e",
                "   finally:",
                "       with open('{result_filename}', 'wb') as f_out:",
                "           pickle.dump((result, exception), f_out)",
                "",
            ]
        ).format(
            func_filename = os.path.join(self._container_workdir, func_filename),
            result_filename = os.path.join(self._container_workdir, result_filename)
        )


    def execute(self,
        function: TransportableObject,
        args: List,
        kwargs: List,
        call_before: List,
        call_after: List,
        dispatch_id: str,
        results_dir: str,
        node_id: int - 1
    ):
        """
            Execute the callable inside the respective container and return the result
    
            Args:
                :param Callable function: The input python function to be executed inside the container whose result is to be returned back
                :param List args: List of positional arguments required by the function
                :param Dict kwargs: Dictionary of keyword arguments required by the function
                :param List call_before: List of callables that will be invoked before the actual function execution
                :param List  call_after: List of callables to be invoked after the function has finished execution
                :param str dispatch_id: The unique identifier of the parent workflow that this electron is a part of
                :param str results_dir: The location where the results from the function execution ought to be saved. This will be mounted inside the container
                :param int node_id: ID of the node in the transport graph which is using this executor
        """
    
        # set the necessary variables such as filenames to be used as part of the execution.
        # Use the dispatch_id and the node_id in order to uniquely identify each file/container being
        # created during execution
        dispatch_info = DispatchInfo(dispatch_id=dispatch_id)
        result_filename = f"result-{dispatch_id}-{node_id}.pkl"
        container_name = f"container-{dispatch_id}-{node_id}"
        execute_script_name = f"dkrexec-{dispatch_id}-{node_id}.py"
    
        with self.get_dispatch_context(dispatch_info), tempfile.NamedTemporaryFile(
                dir=self.workdir) as f, tempfile.NamedTemporaryFile(dir=self.workdir, mode="w") as g:
    
                # Wrap the `function` between the call_before and call_after hooks
                new_args = [function, call_before, call_after]
                for arg in args:
                    new_args.append(arg)
    
                pickle.dump((wrapper_fn, new_args, kwargs), f)
                f.flush()
    
                # Format the command to be executed inside the container
                func_filename = f"func-{dispatch_id}-{node_id}.pkl"
                shutil.copy(f.name, os.path.join(self.workdir, func_filename))
    
                cmd = self.format_exec_command(func_filename, result_filename)
                g.write(cmd)
                g.flush()
                shutil.copy(g.name, os.path.join(self.workdir, execute_script_name))
    
                # Start the container in detached mode
                subprocess.run(
                    [
                        "docker",
                        "container",
                        "run",
                        "-dit",
                         "--rm",
                        "--mount",
                        f"type=bind,source={self.workdir},target={self._container_workdir}",
                        "--name",
                        container_name,
                        self.image
                    ],
                    check=True,
                    capture_output=True,
                )
    
                # Execute the script/command inside the container
                subprocess.run(
                    [
                        "docker",
                        "container",
                        "exec",
                        container_name,
                        "python",
                        f"{self._container_workdir}/{execute_script_name}"
                    ],
                    check=True,
                    capture_output=True
                )
    
                # Assert that a result object was created
                assert os.path.exists(os.path.join(self.workdir, result_filename))
    
                # Read the generated result object from the result pickle file
                with open(os.path.join(self.workdir, result_filename), "rb") as f_read:
                    result, exception = pickle.load(f_read)
                    if exception:
                        raise exception
    
                # Terminate the container
                subprocess.run(
                    [
                        "docker",
                        "container",
                        "stop",
                        container_name
                    ],
                    check=True,
                    capture_output=True
                )
    
        # Return
        return result, exception, None
```

This executor can now be used as part of workflows to execute electons inside Docker containers after being properly imported. Following is a quick example

```python
import os
import covalent as ct
from covalent.executor import DockerExecutor

docker = DockerExecutor(image='docker-executor-demo:latest', workdir=os.getcwd())

@ct.electron(executor=docker)
def add(x, y):
    return x + y

@ct.electron(executor=docker)
def multiply(x, y):
    return x*y


@ct.lattice
def workflow(x, y):
    r1 = add(x, y)
    return multiply(y, r1)

result = ct.dispatch_sync(workflow)(2, 3)
print(result)


Lattice Result
==============
status: COMPLETED
result: 15
inputs: {'args': [2, 3], 'kwargs': {}}
error: None

start_time: 2022-07-18 16:41:48.106224+00:00
end_time: 2022-07-18 16:41:55.053115+00:00

results_dir: results
dispatch_id: 865fb0fd-7be3-4d15-a24d-97e5327b4253

Node Outputs
------------
add(0): 5
:parameter:2(1): 2
:parameter:3(2): 3
multiply(3): 15
:parameter:3(4): 3
```